In [1]:
%run base.ipynb

<h1>System Inertia Matrix example</h1>

In [2]:
inertia_mat = uv_body.body_inertia_matrix()
M_func = Function('M_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m], [inertia_mat]) # for numerical & symbolic use

In [3]:
# example usage of inertia mat
M_mat = np.array(M_func(blue.m, blue.Io, blue.rg[2], blue.added_m, blue.coupl_added_m))
M_mat

array([[17.  ,  0.  ,  0.  ,  0.  ,  0.23,  0.  ],
       [ 0.  , 24.2 ,  0.  , -0.23,  0.  ,  0.  ],
       [ 0.  ,  0.  , 26.07,  0.  ,  0.  ,  0.  ],
       [ 0.  , -0.23,  0.  ,  0.28,  0.  ,  0.  ],
       [ 0.23,  0.  ,  0.  ,  0.  ,  0.28,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.28]])

In [4]:
M_mat.T == M_mat

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

In [5]:
inertia_ned_mat = uv_ned.ned_euler_inertia_matrix()
Onb = vertcat(thet, phi, psi)
M_ned_func = Function('M_ned', [m, I_o, z_g, Onb, decoupled_added_m, coupled_added_m], [inertia_ned_mat]) # for numerical & symbolic use

In [6]:
# example usage of inertia mat in ned
Onb_v = np.array([0.13,0.17,0.1])
np.array(M_ned_func(blue.m, blue.Io, blue.rg[2], Onb_v,  blue.added_m, blue.coupl_added_m))

array([[ 1.72303360e+01, -7.45090577e-01,  1.18405662e+00,
         1.76115723e-02,  2.23648786e-01,  3.57838329e-02],
       [-7.45090577e-01,  2.41747103e+01, -1.91929773e-01,
        -2.26055609e-01,  2.24397276e-02,  3.31239544e-02],
       [ 1.18405662e+00, -1.91929773e-01,  2.58649536e+01,
        -3.85835972e-02, -2.93860503e-02,  0.00000000e+00],
       [ 1.76115723e-02, -2.26055609e-01, -3.85835972e-02,
         2.80000000e-01,  0.00000000e+00, -3.62975599e-02],
       [ 2.23648786e-01,  2.24397276e-02, -2.93860503e-02,
         0.00000000e+00,  2.80000000e-01,  0.00000000e+00],
       [ 3.57838329e-02,  3.31239544e-02,  0.00000000e+00,
        -3.62975599e-02,  0.00000000e+00,  2.80000000e-01]])

<h1>Coriolis Matrix example</h1>

In [7]:
coriolis_mat = uv_body.body_coriolis_centripetal_matrix()
C_func = Function('C_b', [m, I_o, z_g, decoupled_added_m, coupled_added_m, x_nb, v_c], [coriolis_mat]) # for numerical & symbolic use

In [8]:
C = np.array(C_func(blue.m, blue.Io, blue.rg[2], blue.added_m, blue.coupl_added_m, np.array([0.1 ,0.2,0.1,0.1,0.1,0.5]), 
                    np.array([0.0 ,0.0,0.0,0.0,0.0,0.0])))

In [9]:
-C.T == C

array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

<h1>Inverse Dynamics for featherson dynamics </h1>

In [10]:
uv_id = uv_body.body_inverse_dynamics()
uv_id_plucker = vertcat(uv_id[3:], uv_id[:3]) # base vehicle body force to plucker basis used by featherson

x_nb_plucka = vertcat(x_nb[3:], x_nb[:3])
dx_nb_plucka = vertcat(dx_nb[3:], dx_nb[:3])
f_rhs_func = Function('UV_id', [z, eul, x_nb_plucka, dx_nb_plucka, sim_p], [uv_id_plucker]) # for numerical & symbolic use
f_rhs_func.save('float_base.casadi')

<h1>Hacky method to get Inertia, H and bias Froce, B from inverse dynamics for featherson dynamics </h1>

In [11]:
# bias force or (coriolis plus restoring force)
B_F = f_rhs_func(blue.below_surface, eul, x_nb_plucka, [0,0,0,0,0,0], blue.sim_params) 
B_F.size()

(6, 1)

In [12]:
B_F_func = Function('B_F', [x_nb_plucka, eul], [B_F]) # for numerical & symbolic use

In [13]:
B_F_func([0,0,0,0,0,0],[0,0,0])

DM([0, 0, 0, 0, 0, 1.985])

In [14]:
# validate B_F
B_full = uv_body.body_coriolis_centripetal_matrix()@v_r + uv_body.body_damping_matrix()@(v_r) + uv_body.body_restoring_vector()
B_full = vertcat(B_full[3:], B_full[:3]) # base vehicle body force to plucker basis used by featherson
B_full_func = Function('B_full', [z, eul, x_nb, dx_nb, sim_p], [B_full])

B_simpl = B_full_func(blue.below_surface, eul, x_nb, dx_nb, blue.sim_params)
B_simpl_func = Function('B_simpl', [eul, x_nb_plucka, dx_nb_plucka], [B_simpl]) # for numerical & symbolic use
B_simpl_func([0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0])

DM([0, 0, 0, 0, 0, 1.985])

In [15]:
# inertia matrix
tau = f_rhs_func(blue.below_surface, eul, x_nb_plucka, dx_nb_plucka, blue.sim_params)

n = x_nb_plucka.size1()
ID_delta = tau - B_F
H = SX(n, n)
for i in range(0, n):
    d_a = [0]*n
    d_a[i] = 1
    H[:, i] = substitute(ID_delta, dx_nb_plucka, d_a)
H_func = Function('H', [eul, x_nb_plucka], [H]) # for numerical & symbolic use

In [16]:
H_mat = H_func([0,0,0], [0,0,0,0,0,0])
print(H_mat.full())

# validating matrix
H_mat = vertcat(H_mat[3:,:], H_mat[:3,:])
H_mat = horzcat(H_mat[:,3:], H_mat[:,:3]).full()
H_mat == M_mat

[[ 0.28  0.    0.    0.   -0.23  0.  ]
 [ 0.    0.28  0.    0.23  0.    0.  ]
 [ 0.    0.    0.28  0.    0.    0.  ]
 [ 0.    0.23  0.   17.    0.    0.  ]
 [-0.23  0.    0.    0.   24.2   0.  ]
 [ 0.    0.    0.    0.    0.   26.07]]


array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

<h1>Restoring Forces</h1>

In [19]:
res_G = uv_body.body_restoring_vector()
res_G_func = Function('G_n', [z, eul, W, B, r_g, r_b], [res_G]) # for numerical & symbolic use